In [234]:
import requests
from bs4 import BeautifulSoup, Tag
from dataclasses import dataclass
from tqdm import tqdm
import pandas as pd
import os

In [18]:
f = lambda x: (temp := x * 2) + 1 if x > 5 else (temp := x + 3) * 2

In [348]:
def get_headers(referer: str = "https://github.com/topics", xml: bool = True):
    # return {
    #     "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8",
    #     "Accept-Encoding": "gzip, deflate, br, zstd",
    #     "Accept-Language":"en-US,en;q=0.9",
    #     "Priority": "u=1, i",
    #     "Referer": referer,
    #     "Sec-CH-UA": '"Not A(Brand";v="8", "Chromium";v="132", "Brave";v="132"',
    #     "Sec-CH-UA-Mobile": "?0",
    #     "Sec-CH-UA-Platform": '"Windows"',
    #     "Sec-Fetch-Dest": "empty",
    #     "Sec-Fetch-Mode": "cors",
    #     "Sec-Fetch-Site": "same-origin",
    #     "Sec-GPC": "1",
    #     "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36",
    #     "X-Requested-With": "XMLHttpRequest" if xml else None
    # }
    return {
        "accept": "text/html",
        "accept-encoding": "gzip, deflate, br, zstd",
        "accept-language": "en-US,en;q=0.6",
        "priority": "u=1, i",
        "referer": referer,
        "sec-ch-ua": '"Not A(Brand";v="8", "Chromium";v="132", "Brave";v="132"',
        "sec-ch-ua-mobile": "?0",
        "sec-ch-ua-platform": '"Windows"',
        "sec-fetch-dest": "empty",
        "sec-fetch-mode": "cors",
        "sec-fetch-site": "same-origin",
        "sec-gpc": "1",
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36",
        "x-requested-with": "XMLHttpRequest" if xml else None
    }

cookies = None
def get(url: str="https://github.com", xml: bool = False) -> requests.Response:
    global cookies
    r = requests.get(url, headers=get_headers(referer=url, xml=xml), cookies=cookies)
    r.raise_for_status()
    if cookies is None: cookies = r.cookies
    return r

In [16]:
r = get("https://github.com/topics?page=1", xml=True)
soup = BeautifulSoup(r.content)

In [145]:
from typing import Callable, TypeVar, List

T = TypeVar('T')  # Generic type for elements in the list
U = TypeVar('U')  # Generic type for the result of the map function

class List2(list[T]):
    def map(self, func: Callable[[T], U]) -> 'List2[U]':
        return List2(map(func, self))  # Apply func to each element of the list

    def filter(self, func: Callable[[T], bool]) -> 'List2[T]':
        return List2(filter(func, self))  # Keep only elements where func returns True

    @classmethod
    def from_list(cls, lst: List[T]) -> 'List2[T]':
        return cls(lst)  # Create a List2 instance from a regular list

In [87]:
@dataclass
class Topic:
    href: str
    title: str
    body: str
    def to_json(self) -> dict:
        return {
            'href': self.href,
            'title': self.title,
            'body': self.body
        }
    
def get_topics(soup: BeautifulSoup) -> List2[Topic]:
    items: List2[Tag] = List2.from_list(
        soup
        .find('html', recursive=False)
        .find('body', recursive=False)
        .find('div', recursive=False)
        .find_all(recursive=False)
    )
    
    def get_from_item(item: Tag) -> Topic:
        item: List2[Tag] = List2.from_list(
            item.find_all('a', recursive=True)
        )
        return item.filter(lambda y: (
            y.find('p', recursive=False) is not None
        )).map(lambda y: Topic(
            href= y.get('href'),
            title= y.find_all('p', recursive=False)[0].text,
            body= y.find_all('p', recursive=False)[1].text.strip()
        ))[0]
    
    return items.map(get_from_item)

get_topics(soup)[:1]

[Topic(href='/topics/3d', title='3D', body='3D refers to the use of three-dimensional graphics, modeling, and animation in various industries.')]

In [ ]:
topics = []

def featch_topic(page: int = 1):
    global topics
    idx = page - 1
    assert len(topics) >= idx, "Something went wrong"
    if len(topics) == idx:
        r = get(f"https://github.com/topics?page={page}", xml=True)
        soup = BeautifulSoup(r.content)
        topics.append(get_topics(soup))
    return topics[idx]

In [111]:
for i in tqdm(range(1, 10)):
    featch_topic(i)
    
filtered_topics: List2[Topic] = List2.from_list([j for i in topics for j in i])
filtered_topics

100%|██████████| 9/9 [00:00<?, ?it/s]


[Topic(href='/topics/3d', title='3D', body='3D refers to the use of three-dimensional graphics, modeling, and animation in various industries.'),
 Topic(href='/topics/ajax', title='Ajax', body='Ajax is a technique for creating interactive web applications.'),
 Topic(href='/topics/algorithm', title='Algorithm', body='Algorithms are self-contained sequences that carry out a variety of tasks.'),
 Topic(href='/topics/amphp', title='Amp', body='Amp is a non-blocking concurrency library for PHP.'),
 Topic(href='/topics/android', title='Android', body='Android is an operating system built by Google designed for mobile devices.'),
 Topic(href='/topics/angular', title='Angular', body='Angular is an open source web application platform.'),
 Topic(href='/topics/ansible', title='Ansible', body='Ansible is a simple and powerful automation engine.'),
 Topic(href='/topics/api', title='API', body='An API (Application Programming Interface) is a collection of protocols and subroutines for building soft

In [ ]:
df_topics = pd.DataFrame(filtered_topics.map(lambda x: x.to_json()))
df_topics.to_json('topics.jsonl', orient='records', lines=True)

In [120]:
df_topics = pd.read_json('topics.jsonl', lines=True)
df_topics

,href,title,body
0,/topics/3d,3D,3D refers to the use of three-dimensional grap...
1,/topics/ajax,Ajax,Ajax is a technique for creating interactive w...
2,/topics/algorithm,Algorithm,Algorithms are self-contained sequences that c...
3,/topics/amphp,Amp,Amp is a non-blocking concurrency library for ...
4,/topics/android,Android,Android is an operating system built by Google...
...,...,...,...
160,/topics/twitter,Twitter,Twitter is an online news and social networkin...
161,/topics/typescript,TypeScript,TypeScript is a typed superset of JavaScript t...
162,/topics/ubuntu,Ubuntu,Ubuntu is a Linux-based operating system.
163,/topics/unity,Unity,Unity is a game engine used to create 2D/3D vi...


In [146]:
topic = df_topics.iloc[0]
topic

href                                            /topics/3d
title                                                   3D
body     3D refers to the use of three-dimensional grap...
Name: 0, dtype: object

In [203]:
r = get(f"https://github.com/{topic.href}?page={1}", xml=True)
soup = BeautifulSoup(r.content)

In [254]:
@dataclass
class Repo:
    repo_owner: str | None
    repo_name: str | None
    repo_url: str
    start: int
    icon: str
    desc: str | None
    tags: List2[tuple[str | None, str]]
    timestamp: str
    lang: str | None
    def to_json(self) -> dict:
        return {
                'repo_owner': self.repo_owner,
                'repo_name': self.repo_name,
                'repo_url': self.repo_url,
                'start': self.start,
                'icon': self.icon,
                'desc': self.desc,
                'tags': self.tags,
                'timestamp': self.timestamp,
                'lang': self.lang
            }
    

def get_topics_repo(soup: BeautifulSoup) -> List2[Repo]:
    items: List2[Tag] = List2.from_list(
        soup
        .find('html', recursive=False)
        .find('body', recursive=False)
        .find_all('article', recursive=False)    
    )
    
    def get_text(tag: None | Tag) -> str | None:
        if tag is None: return None
        return tag.text
    
    def get_from_item(item: Tag) -> Repo | None:
        start_count = item.find('span', {'id': "repo-stars-counter-star"}, recursive=True).get('title')
        title_div = item.find('h3', recursive=True).find_all('a', recursive=False)
        icon_div = item.find('a', recursive=False)
        main_div = item.find_all('div', recursive=False)[1].find_all('div', recursive=False)
        if len(main_div) < 3: return None
        desc_div, tags_div, timestamp_div = main_div
        return Repo(
            repo_owner = get_text(title_div[0]),
            repo_name = get_text(title_div[1]),
            repo_url = title_div[1].get('href'),
            start = int(start_count.replace(',', '').strip()),
            icon = icon_div.find('img', recursive=False).get('src') if icon_div else None,
            desc = get_text(desc_div.find('p', recursive=False)),
            tags = List2.from_list(tags_div.find_all('a')).map(lambda x: (get_text(x), x.get('href'))),
            timestamp = timestamp_div.find('relative-time').get('datetime'),
            lang = get_text(timestamp_div.find('span', {'itemprop': 'programmingLanguage'}))
        )
    
    return items.map(get_from_item).filter(lambda x: x is not None)

repos = get_topics_repo(soup)

In [284]:
repos = {
    
}

In [343]:
def featch_topic_repo(topic: Topic, page: int = 1, force: bool = True) -> tuple[bool, List2[Repo]]:
    global repos
    idx = page - 1
    
    cache = False
        
    if repos.get(topic.title) is None:
        r = get(f"https://github.com/{topic.href}?page={page}", xml=True)
        soup = BeautifulSoup(r.content)
        repos[topic.title] = [get_topics_repo(soup)]
        # repos.append(get_topics_repo(soup)) # TODO Multipage repo
    else:   
        assert len(repos[topic.title]) >= idx, "Something went wrong"
        if len(repos[topic.title]) == idx:
            r = get(f"https://github.com/{topic.href}?page={page}", xml=True)
            soup = BeautifulSoup(r.content)
            repos[topic.title].append(get_topics_repo(soup))
        else: cache = True
    if force and len(repos[topic.title][idx]) == 0:
        del repos[topic.title][idx]
        return featch_topic_repo(topic=topic, page=page, force=False)
    return cache, repos[topic.title][idx]

In [332]:
featch_topic_repo(topic=topic, page=4)

(True,
 [Repo(repo_owner='DarklightGames', repo_name='io_scene_psk_psa', repo_url='/DarklightGames/io_scene_psk_psa', start=428, icon=None, desc='A Blender extension for importing and exporting Unreal PSK and PSA files', tags=[('python', '/topics/python'), ('export', '/topics/export'), ('psa', '/topics/psa'), ('blender', '/topics/blender'), ('import', '/topics/import'), ('unreal-engine', '/topics/unreal-engine'), ('blender-addon', '/topics/blender-addon'), ('psk', '/topics/psk'), ('blender-extension', '/topics/blender-extension')], timestamp='2025-01-21T03:58:00Z', lang='Python'),
  Repo(repo_owner='spatialos', repo_name='UnrealGDK', repo_url='/spatialos/UnrealGDK', start=406, icon='https://repository-images.githubusercontent.com/109980189/3aa77880-e8fb-11e9-918f-fb5b99ac19f6', desc='SpatialOS GDK for Unreal Plugin', tags=[('online', '/topics/online'), ('gdk', '/topics/gdk'), ('mmo', '/topics/mmo'), ('unreal-engine', '/topics/unreal-engine'), ('improbable', '/topics/improbable'), ('spa

In [352]:
with tqdm(total=len(filtered_topics) * 10) as pbar:
    for topic in filtered_topics:
        flag = False
        for i in range(10):
            pbar.update(1)
            try:    
                cache, item = featch_topic_repo(topic=topic, page=i, force=True)
            except Exception as e: item = None 
            if item is None: 
                if flag: 
                    print(f'{topic} [{i}] Skipping...')
                    break
                else: flag = True
                continue
        df_repos = pd.DataFrame(List2.from_list([ii for i in repos[topic.title] for ii in i]).map(lambda x: x.to_json()))
        df_repos.to_json(os.path.join('repos', f'{topic.title}.jsonl'), orient='records', lines=True)

 63%|██████▎   | 1038/1650 [03:51<00:53, 11.41it/s]

Topic(href='/topics/mvvmcross', title='MvvmCross', body='The .NET MVVM framework for cross-platform solutions, including Xamarin.iOS, Xamarin.Android, Windows, and Mac.') [7] Skipping...


 80%|███████▉  | 1316/1650 [07:41<08:33,  1.54s/it]

Topic(href='/topics/ratchet', title='Ratchet', body='Ratchet is a set of libraries to handle WebSockets asynchronously in PHP.') [7] Skipping...


 90%|████████▉ | 1483/1650 [13:11<03:44,  1.34s/it]

Topic(href='/topics/spacevim', title='SpaceVim', body='SpaceVim is a community-driven distribution of the vim editor that allows managing your plugins in layers.') [6] Skipping...


100%|█████████▉| 1643/1650 [18:27<00:04,  1.48it/s]


In [350]:
# repos['Qt']
featch_topic_repo(topic=filtered_topics.filter(lambda x: x.title == 'Qt')[0], page=2, force=True)

(False,
 [Repo(repo_owner='visualfc', repo_name='liteide', repo_url='/visualfc/liteide', start=7641, icon=None, desc='LiteIDE is a simple, open source, cross-platform Go IDE. ', tags=[('go', '/topics/go'), ('golang', '/topics/golang'), ('qt', '/topics/qt'), ('cross-platform', '/topics/cross-platform'), ('ide', '/topics/ide'), ('liteide', '/topics/liteide')], timestamp='2024-08-02T02:23:07Z', lang='C++'),
  Repo(repo_owner='albertlauncher', repo_name='albert', repo_url='/albertlauncher/albert', start=7364, icon=None, desc='A fast and flexible keyboard launcher', tags=[('python', '/topics/python'), ('qt', '/topics/qt'), ('cpp', '/topics/cpp'), ('launcher', '/topics/launcher'), ('plugins', '/topics/plugins'), ('application-launcher', '/topics/application-launcher'), ('albert', '/topics/albert'), ('albertlauncher', '/topics/albertlauncher'), ('keyboard-launcher', '/topics/keyboard-launcher'), ('qt6', '/topics/qt6')], timestamp='2025-01-06T22:22:29Z', lang='C++'),
  Repo(repo_owner='PrismLa

In [342]:
repos['Qt']

[[]]

In [354]:
df_all_repos = pd.DataFrame([i.to_json() for _, i in dict([(vv.repo_url, vv) for k, v in repos.items() for vv in [xx for x in v for xx in x]]).items()])
df_all_repos.to_json('repos.jsonl', orient='records', lines=True)

In [321]:
len([(vv.repo_url, vv) for k, v in repos.items() for vv in [xx for x in v for xx in x]])

9482

In [362]:
import pickle
with open('repos.pkl', 'wb') as f: 
    pickle.dump([[x for x in i] for i in repos], f)

In [361]:
import pickle
with open('topics.pkl', 'wb') as f: 
    pickle.dump([[x for x in i] for i in topics], f)

In [317]:
[vv for v in repos['3D'] for vv in v]

[Repo(repo_owner='mrdoob', repo_name='three.js', repo_url='/mrdoob/three.js', start=103960, icon='https://repository-images.githubusercontent.com/576201/6c52fa00-6238-11eb-8763-f36f6e226bba', desc='JavaScript 3D Library.', tags=[('javascript', '/topics/javascript'), ('svg', '/topics/svg'), ('webgl', '/topics/webgl'), ('html5', '/topics/html5'), ('canvas', '/topics/canvas'), ('augmented-reality', '/topics/augmented-reality'), ('webaudio', '/topics/webaudio'), ('virtual-reality', '/topics/virtual-reality'), ('webgl2', '/topics/webgl2'), ('3d', '/topics/3d'), ('webgpu', '/topics/webgpu'), ('webxr', '/topics/webxr')], timestamp='2025-01-23T10:19:50Z', lang='JavaScript'),
 Repo(repo_owner='pmndrs', repo_name='react-three-fiber', repo_url='/pmndrs/react-three-fiber', start=28037, icon=None, desc='ðŸ‡¨ðŸ‡\xad A React renderer for Three.js', tags=[('react', '/topics/react'), ('threejs', '/topics/threejs'), ('animation', '/topics/animation'), ('renderer', '/topics/renderer'), ('fiber', '/topics